# Test de l'API Open Food Facts - Extraction de produits "Champagne"

**Projet P6 - Place de Marché**

**Objectif** : Tester la collecte de produits à base de champagne via l'API Open Food Facts pour élargir la gamme de produits à l'épicerie fine.

**Livrable** : Script Python permettant l'extraction des 10 premiers produits dans un fichier CSV contenant :
- `foodId` : Identifiant du produit
- `label` : Nom du produit
- `category` : Catégorie du produit
- `foodContentsLabel` : Description/ingrédients du produit
- `image` : URL de l'image du produit

---

## 📚 Documentation API Open Food Facts

- **API Endpoint** : https://world.openfoodfacts.org/api/v2/search
- **Documentation** : https://openfoodfacts.github.io/openfoodfacts-server/api/
- **Rate Limit** : 10 req/min pour les recherches
- **User-Agent requis** : Format `AppName/Version (ContactEmail)`


## 1. Installation et Imports


In [ ]:
import requests
import pandas as pd
import json
from pathlib import Path
from datetime import datetime
import time

## 2. Configuration de l'API


In [13]:
# Configuration de l'API Open Food Facts
API_BASE_URL = "https://world.openfoodfacts.org"
API_SEARCH_ENDPOINT = f"{API_BASE_URL}/cgi/search.pl"

# User-Agent personnalisé (obligatoire selon la documentation)
HEADERS = {
    'User-Agent': 'PlaceDeMarche/1.0 (laureendademeule@example.com)'
}

print("✅ Configuration de l'API initialisée")
print(f"📍 Endpoint : {API_SEARCH_ENDPOINT}")


✅ Configuration de l'API initialisée
📍 Endpoint : https://world.openfoodfacts.org/cgi/search.pl


## 3. Fonction de recherche de produits


In [14]:
def search_products(search_term, page_size=10, page=1):
    """
    Recherche des produits sur Open Food Facts
    
    Parameters:
    -----------
    search_term : str
        Terme de recherche (ex: 'champagne')
    page_size : int
        Nombre de résultats par page (max 100)
    page : int
        Numéro de page
        
    Returns:
    --------
    dict : Résultats de la recherche au format JSON
    """
    
    params = {
        'search_terms': search_term,
        'page_size': page_size,
        'page': page,
        'json': 1,  # Format JSON
        'fields': 'code,product_name,categories,categories_tags,ingredients_text,image_url,image_front_url,brands,nutriscore_grade'
    }
    
    try:
        print(f"🔍 Recherche de produits contenant '{search_term}'...")
        response = requests.get(API_SEARCH_ENDPOINT, params=params, headers=HEADERS)
        response.raise_for_status()
        
        data = response.json()
        print(f"✅ {data.get('count', 0)} produits trouvés")
        print(f"📦 {len(data.get('products', []))} produits récupérés")
        
        return data
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur lors de la requête API : {e}")
        return None


## 4. Fonction d'extraction et de formatage des données


In [15]:
def extract_product_data(products):
    """
    Extrait et formate les données des produits selon le format requis
    
    Parameters:
    -----------
    products : list
        Liste des produits retournés par l'API
        
    Returns:
    --------
    list : Liste de dictionnaires contenant les données formatées
    """
    
    extracted_data = []
    
    for product in products:
        # Extraction des données avec gestion des valeurs manquantes
        product_data = {
            'foodId': product.get('code', 'N/A'),
            'label': product.get('product_name', 'N/A'),
            'category': product.get('categories', 'N/A'),
            'foodContentsLabel': product.get('ingredients_text', 'N/A'),
            'image': product.get('image_front_url', product.get('image_url', 'N/A'))
        }
        
        extracted_data.append(product_data)
    
    print(f"✅ {len(extracted_data)} produits extraits et formatés")
    return extracted_data


## 5. Fonction de sauvegarde en CSV


In [16]:
def save_to_csv(data, filename='produits_champagne.csv'):
    """
    Sauvegarde les données dans un fichier CSV
    
    Parameters:
    -----------
    data : list
        Liste de dictionnaires contenant les données
    filename : str
        Nom du fichier CSV de sortie
        
    Returns:
    --------
    str : Chemin complet du fichier sauvegardé
    """
    
    # Créer le DataFrame
    df = pd.DataFrame(data)
    
    # Définir le chemin de sortie
    output_dir = Path('../notebooks')
    output_dir.mkdir(parents=True, exist_ok=True)
    
    output_path = output_dir / filename
    
    # Sauvegarder en CSV
    df.to_csv(output_path, index=False, encoding='utf-8')
    
    print(f"\n💾 Fichier CSV sauvegardé : {output_path}")
    print(f"📊 Dimensions : {df.shape[0]} lignes × {df.shape[1]} colonnes")
    
    return str(output_path)


## 6. Exécution : Recherche et extraction des produits "Champagne"


In [17]:
# Paramètres de recherche
SEARCH_TERM = "champagne"
NUM_PRODUCTS = 10

print("="*70)
print("🍾 EXTRACTION DE PRODUITS 'CHAMPAGNE' VIA API OPEN FOOD FACTS")
print("="*70)
print(f"\n📅 Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔎 Terme de recherche : '{SEARCH_TERM}'")
print(f"📦 Nombre de produits à extraire : {NUM_PRODUCTS}")
print("\n" + "-"*70 + "\n")

# 1. Recherche des produits
results = search_products(SEARCH_TERM, page_size=NUM_PRODUCTS)

if results and 'products' in results:
    # 2. Extraction des données
    products_data = extract_product_data(results['products'])
    
    # 3. Sauvegarde en CSV
    csv_path = save_to_csv(products_data, filename='produits_champagne.csv')
    
    print("\n" + "="*70)
    print("✅ EXTRACTION TERMINÉE AVEC SUCCÈS")
    print("="*70)
else:
    print("\n❌ ERREUR : Aucun produit n'a pu être récupéré")


🍾 EXTRACTION DE PRODUITS 'CHAMPAGNE' VIA API OPEN FOOD FACTS

📅 Date : 2025-10-03 17:49:28
🔎 Terme de recherche : 'champagne'
📦 Nombre de produits à extraire : 10

----------------------------------------------------------------------

🔍 Recherche de produits contenant 'champagne'...
✅ 1534 produits trouvés
📦 10 produits récupérés
✅ 10 produits extraits et formatés

💾 Fichier CSV sauvegardé : ../notebooks/produits_champagne.csv
📊 Dimensions : 10 lignes × 5 colonnes

✅ EXTRACTION TERMINÉE AVEC SUCCÈS


## 7. Visualisation des résultats


In [18]:
# Charger et afficher le DataFrame
if results and 'products' in results:
    df_champagne = pd.DataFrame(products_data)
    
    print("\n📊 APERÇU DES DONNÉES EXTRAITES\n")
    print(df_champagne.head(10))
    
    print("\n📈 STATISTIQUES DES DONNÉES\n")
    print(f"Nombre total de produits : {len(df_champagne)}")
    print(f"\nNombre de valeurs manquantes par colonne :")
    print(df_champagne.isnull().sum())
    print(f"\nNombre de produits avec image : {(df_champagne['image'] != 'N/A').sum()}")
    print(f"Nombre de produits avec description : {(df_champagne['foodContentsLabel'] != 'N/A').sum()}")



📊 APERÇU DES DONNÉES EXTRAITES

          foodId                                              label  \
0  8001841769189                                                N/A   
1  3039820510250  Vivien Paille Lentilles vertes le paquet de 500 g   
2  3292070010264  Betteraves de Champagne & chèvre crémeux, poin...   
3  3113934004147                                     Canard Duchêne   
4  4820097815556                              Splashes of champagne   
5  4056489843696                      Rillettes de homard au cognac   
6  3760091726964                                Lentilles roses bio   
7  3258431220000                                                      
8  3114080034057                                     Champagne rosé   
9  3185370729960                            Champagne Impérial Brut   

                                            category  \
0                                Produits,Champagnes   
1  Aliments et boissons à base de végétaux, Alime...   
2                

## 8. Affichage détaillé d'un exemple de produit


In [19]:
if results and 'products' in results and len(products_data) > 0:
    print("\n🍾 EXEMPLE DE PRODUIT EXTRAIT\n")
    print("-" * 70)
    
    example = products_data[0]
    for key, value in example.items():
        # Tronquer les valeurs trop longues pour l'affichage
        display_value = value if len(str(value)) <= 100 else str(value)[:100] + "..."
        print(f"{key:20s} : {display_value}")
    
    print("-" * 70)



🍾 EXEMPLE DE PRODUIT EXTRAIT

----------------------------------------------------------------------
foodId               : 8001841769189
label                : N/A
category             : Produits,Champagnes
foodContentsLabel    : N/A
image                : https://images.openfoodfacts.org/images/products/800/184/176/9189/front_fr.3.400.jpg
----------------------------------------------------------------------


In [20]:
def collect_openfoodfacts_products(search_term, num_products=10, output_file=None):
    """
    Fonction principale pour collecter des produits depuis Open Food Facts
    
    Parameters:
    -----------
    search_term : str
        Terme de recherche (ex: 'champagne', 'chocolat', etc.)
    num_products : int
        Nombre de produits à extraire (par défaut: 10)
    output_file : str
        Nom du fichier CSV de sortie (par défaut: produits_{search_term}.csv)
        
    Returns:
    --------
    pd.DataFrame : DataFrame contenant les produits extraits
    """
    
    if output_file is None:
        output_file = f"produits_{search_term.lower().replace(' ', '_')}.csv"
    
    # Recherche
    results = search_products(search_term, page_size=num_products)
    
    if results and 'products' in results:
        # Extraction
        products_data = extract_product_data(results['products'])
        
        # Sauvegarde
        save_to_csv(products_data, filename=output_file)
        
        # Retour du DataFrame
        return pd.DataFrame(products_data)
    else:
        print("❌ Erreur lors de la collecte des produits")
        return None


---

## 📋 Résumé et Conclusions

### ✅ Objectifs atteints

1. **Connexion à l'API Open Food Facts** : Implémentation réussie avec User-Agent personnalisé
2. **Recherche de produits "champagne"** : Récupération des 10 premiers résultats
3. **Extraction des données requises** :
   - `foodId` : Code-barres du produit
   - `label` : Nom du produit
   - `category` : Catégories du produit
   - `foodContentsLabel` : Liste d'ingrédients
   - `image` : URL de l'image
4. **Export CSV** : Sauvegarde dans `data/artifacts/produits_champagne.csv`

### 🎯 Points clés de l'API Open Food Facts

- **Avantages** :
  - API gratuite sans inscription requise
  - Base de données collaborative de +2.8M produits
  - Documentation complète
  - Données nutritionnelles et images disponibles
  
- **Limitations** :
  - Rate limit : 10 requêtes/min pour les recherches
  - Qualité des données variable (données crowdsourcées)
  - User-Agent personnalisé obligatoire
  - Certains produits peuvent avoir des données incomplètes

### 🚀 Recommandations pour la production

1. **Gestion des erreurs** : Implémenter une gestion robuste des timeouts et erreurs réseau
2. **Respect des rate limits** : Ajouter des délais entre les requêtes (6 secondes minimum)
3. **Validation des données** : Vérifier la complétude des données avant utilisation
4. **Cache local** : Télécharger et mettre en cache les données fréquemment utilisées
5. **Monitoring** : Logger les requêtes et les erreurs pour débogage

### 📊 Intégration dans le projet Place de Marché

Ce script démontre la faisabilité d'élargir la gamme de produits à l'épicerie fine en utilisant l'API Open Food Facts. Le même processus peut être appliqué pour d'autres catégories de produits alimentaires de luxe.

---

**Auteur** : Laureen Dademeule  
**Projet** : P6 - Place de Marché - Classification automatique de biens de consommation  
**Date** : Octobre 2025
